# Trajectory Recommendation

<a id='toc'></a>

[Table of Contents](#toc)
1. [Preprocess Dataset](#sec1)
  1. [Load Data](#sec1.1)
  1. [Utility function](#sec1.2)
1. [POI Ranking](#sec2)
  1. [POI Features for Ranking](#sec2.1)
  1. [Training DataFrame](#sec2.2)
  1. [Test DataFrame](#sec2.3)
  1. [Ranking POIs using rankSVM](#sec2.4)
  1. [Recommendation based on POI Ranking](#sec2.5)
1. [Factorise Transition Probabilities in POI Feature Space](#sec3)
  1. [POI Features for Factorisation](#sec3.1)
  1. [Transition Matrix between POI Cateogries](#sec3.2)
  1. [Transition Matrix between POI Popularity Classes](#sec3.3)
  1. [Transition Matrix between the Number of POI Visit Classes](#sec3.4)
  1. [Transition Matrix between POI Average Visit Duration Classes](#sec3.5)
  1. [Transition Matrix between POI Neighborhood Classes](#sec3.6)
  1. [Transition Matrix between POIs](#sec3.7)
  1. [Recommendation based on POI Transition Matrix](#sec3.8)
1. [Combine POI Ranking with Factorised Markov Chain](#sec4)
1. [Comparison](#sec5)

<a id='sec1'></a>

# 1. Preprocess Dataset

In [ ]:
% matplotlib inline

import os, sys, time, pickle, tempfile
import math, random, itertools, scipy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as sop
from scipy.linalg import kron
from scipy.misc import logsumexp
from joblib import Parallel, delayed
from sklearn.cluster import KMeans
import seaborn as sns
import pulp

In [ ]:
random.seed(987654321) # control random choice when splitting training/testing set
np.random.seed(987654321)
ranksvm_dir = '$HOME/work/ranksvm'

In [ ]:
data_dir = 'data/data-recsys16'
suffix = ['Osak', 'Glas', 'Edin', 'Toro', 'Melb']

In [ ]:
dat_ix = 2

In [ ]:
noshort = False

In [ ]:
uspecific = False
KX = 100  # 100 folds in user specific setting
kxstr = str(KX) + 'X-'

In [ ]:
ALPHA = 0.5
alphastr = str(ALPHA).replace('.', '_') + '-'

In [ ]:
fpoi = os.path.join(data_dir, 'poi-' + suffix[dat_ix] + '.csv')

In [ ]:
if noshort == True:
    ftraj = os.path.join(data_dir, 'traj-noshort-' + suffix[dat_ix] + '.csv')    
else:
    ftraj = os.path.join(data_dir, 'traj-all-' + suffix[dat_ix] + '.csv')

Files to store results.

In [ ]:
if noshort == True:
    if uspecific == True:
        frecdict_rank = os.path.join(data_dir, 'rank-noshort-specific-' + kxstr + suffix[dat_ix] + '.pkl')
        frecdict_tran = os.path.join(data_dir, 'tran-noshort-specific-' + kxstr + suffix[dat_ix] + '.pkl')
        frecdict_comb = os.path.join(data_dir, 'comb-noshort-specific-' + alphastr + kxstr + suffix[dat_ix] + '.pkl')
    else:
        frecdict_rank = os.path.join(data_dir, 'rank-noshort-agnostic-' + suffix[dat_ix] + '.pkl')
        frecdict_tran = os.path.join(data_dir, 'tran-noshort-agnostic-' + suffix[dat_ix] + '.pkl')
        frecdict_comb = os.path.join(data_dir, 'comb-noshort-agnostic-' + alphastr + suffix[dat_ix] + '.pkl')
else:
    if uspecific == True:
        frecdict_rank = os.path.join(data_dir, 'rank-all-specific-' + kxstr + suffix[dat_ix] + '.pkl')
        frecdict_tran = os.path.join(data_dir, 'tran-all-specific-' + kxstr + suffix[dat_ix] + '.pkl')
        frecdict_comb = os.path.join(data_dir, 'comb-all-specific-' + alphastr + kxstr + suffix[dat_ix] + '.pkl')
    else:
        frecdict_rank = os.path.join(data_dir, 'rank-all-agnostic-' + suffix[dat_ix] + '.pkl')
        frecdict_tran = os.path.join(data_dir, 'tran-all-agnostic-' + suffix[dat_ix] + '.pkl')
        frecdict_comb = os.path.join(data_dir, 'comb-all-agnostic-' + alphastr + suffix[dat_ix] + '.pkl')

Method switches.

In [ ]:
run_rank = True
run_tran = True
run_comb = True

<a id='sec1.1'></a>

## 1.1 Load Data

In [ ]:
traj_all = pd.read_csv(ftraj)
traj_all.head()

In [ ]:
poi_all = pd.read_csv(fpoi)
poi_all.set_index('poiID', inplace=True)
poi_all.head()

In [ ]:
num_user = traj_all['userID'].unique().shape[0]
num_poi = traj_all['poiID'].unique().shape[0]
num_traj = traj_all['trajID'].unique().shape[0]
assert(num_poi == poi_all.shape[0])
pd.DataFrame({'#user': num_user, '#poi': num_poi, '#traj': num_traj, '#traj/user': num_traj/num_user}, \
             index=[str(suffix[dat_ix])])

In [ ]:
ax = traj_all['trajLen'].hist(bins=20)
ax.set_yscale('log')

Mapping trajectory to user.

In [ ]:
traj_user = traj_all[['trajID', 'userID']].copy().groupby('trajID').first()
traj_user.head()

<a id='sec1.2'></a>

## 1.2 Utility function

Print computing progress.

In [ ]:
def print_progress(cnt, total):
    """Display a progress bar"""
    assert(cnt > 0 and total > 0 and cnt <= total)
    length = 80
    ratio = cnt / total
    n = int(length * ratio)
    sys.stdout.write('\r[%-80s] %d%%' % ('-'*n, int(ratio*100)))
    sys.stdout.flush()

Extract trajectory, i.e., a list of POIs.

In [ ]:
def extract_traj(tid, traj_all):
    traj = traj_all[traj_all['trajID'] == tid].copy()
    traj.sort_values(by=['startTime'], ascending=True, inplace=True)
    return traj['poiID'].tolist()

Compute POI properties, e.g., popularity, total number of visit, average visit duration.

In [ ]:
def calc_poi_info(trajid_list, traj_all, poi_all):
    assert(len(trajid_list) > 0)
    # to allow duplicated trajid
    poi_info = traj_all[traj_all['trajID'] == trajid_list[0]][['poiID', 'poiDuration']].copy() 
    for i in range(1, len(trajid_list)):
        traj = traj_all[traj_all['trajID'] == trajid_list[i]][['poiID', 'poiDuration']]
        poi_info = poi_info.append(traj, ignore_index=True)
    
    poi_info = poi_info.groupby('poiID').agg([np.mean, np.size])
    poi_info.columns = poi_info.columns.droplevel()
    poi_info.reset_index(inplace=True)
    poi_info.rename(columns={'mean':'avgDuration', 'size':'nVisit'}, inplace=True)
    poi_info.set_index('poiID', inplace=True) 
    poi_info['poiCat'] = poi_all.loc[poi_info.index, 'poiCat']
    poi_info['poiLon'] = poi_all.loc[poi_info.index, 'poiLon']
    poi_info['poiLat'] = poi_all.loc[poi_info.index, 'poiLat']
    
    # POI popularity: the number of distinct users that visited the POI
    pop_df = traj_all[traj_all['trajID'].isin(trajid_list)][['poiID', 'userID']].copy()
    pop_df = pop_df.groupby('poiID').agg(pd.Series.nunique)
    pop_df.rename(columns={'userID':'nunique'}, inplace=True)
    poi_info['popularity'] = pop_df.loc[poi_info.index, 'nunique']
    
    return poi_info.copy()

Compute the F1 score for recommended trajectory.

In [ ]:
def calc_F1(seq_act, seq_rec):
    '''Compute recall, precision and F1 when trajectories contain sub-tours'''
    assert(len(seq_act) > 0)
    assert(len(seq_rec) > 0)
    #actset = set(seq_act)
    #recset = set(seq_rec)
    #intersect = actset & recset
    
    match_tags = np.zeros(len(seq_act), dtype=np.bool)
    for poi in seq_rec:
        for j in range(len(seq_act)):
            if match_tags[j] == False and poi == seq_act[j]:
                match_tags[j] = True
                break
    intersize = np.nonzero(match_tags)[0].shape[0]
    recall = intersize / len(seq_act)
    precision = intersize / len(seq_rec)
    F1 = 2 * precision * recall / (precision + recall)
    return F1

Compute distance between two POIs using [Haversine formula](http://en.wikipedia.org/wiki/Great-circle_distance).

In [ ]:
def calc_dist(longitude1, latitude1, longitude2, latitude2):
    """Calculate the distance (unit: km) between two places on earth"""
    # convert degrees to radians
    lon1 = math.radians(longitude1)
    lat1 = math.radians(latitude1)
    lon2 = math.radians(longitude2)
    lat2 = math.radians(latitude2)
    radius = 6371.0088 # mean earth radius is 6371.0088km, en.wikipedia.org/wiki/Earth_radius#Mean_radius
    # The haversine formula, en.wikipedia.org/wiki/Great-circle_distance
    dlon = math.fabs(lon1 - lon2)
    dlat = math.fabs(lat1 - lat2)
    return 2 * radius * math.asin(math.sqrt(\
               (math.sin(0.5*dlat))**2 + math.cos(lat1) * math.cos(lat2) * (math.sin(0.5*dlon))**2 ))

Define a *query* (in IR terminology) using tuple (start POI, end POI, #POI) ~~user ID.~~

In [ ]:
trajid_set_all = sorted(traj_all['trajID'].unique().tolist())

In [ ]:
query_id_dict = dict()  # (start, end, length) --> qid

In [ ]:
trajs = [extract_traj(tid, traj_all) for tid in trajid_set_all]
keys = [(t[0], t[-1], len(t)) for t in trajs if len(t) > 2]
cnt = 0
for key in keys:
    if key not in query_id_dict:   # (start, end, length) --> qid
        query_id_dict[key] = cnt
        cnt += 1

In [ ]:
poi_info_ = calc_poi_info(trajid_set_all, traj_all, poi_all)
poi_info_.head()

In [ ]:
print('#traj in total:', len(trajid_set_all))
print('#traj (length > 2):', traj_all[traj_all['trajLen'] > 2]['trajID'].unique().shape[0])
print('#query tuple:', len(query_id_dict))

<a id='sec2'></a>

# 2. POI Ranking

<a id='sec2.1'></a>

## 2.1 POI Features for Ranking

POI Features used for ranking:
1. `popularity`: POI popularity, i.e., the number of distinct users that visited the POI
1. `nVisit`: the total number of visit by all users
1. `avgDuration`: average POI visit duration
1. `sameCatStart`: 1 if POI category is the same as that of `startPOI`, -1 otherwise
1. `sameCatEnd`: 1 if POI category is the same as that of `endPOI`, -1 otherwise
1. `distStart`: distance (haversine formula) from `startPOI`
1. `distEnd`: distance from `endPOI`
1. `seqLen`: trajectory length (copy from query)
1. `diffPopStart`: difference in POI popularity from `startPOI`
1. `diffPopEnd`: difference in POI popularity from `endPOI`
1. `diffNVisitStart`: difference in the total number of visit from `startPOI`
1. `diffNVisitEnd`: difference in the total number of visit from `endPOI`
1. `diffDurationStart`: difference in average POI visit duration from the actual duration spent at `startPOI`
1. `diffDurationEnd`: difference in average POI visit duration from the actual duration spent at `endPOI`

In [ ]:
df_columns = ['poiID', 'label', 'queryID', 'popularity', 'nVisit', 'avgDuration', \
              'sameCatStart', 'sameCatEnd', 'distStart', 'distEnd', 'seqLen', 'diffPopStart', \
              'diffPopEnd', 'diffNVisitStart', 'diffNVisitEnd', 'diffDurationStart', 'diffDurationEnd']

~~Features aggregated from a number of trajectories:~~
~~1. Compute POI `popularity` and average visit `duration` using all trajectories from training and querying set,~~
~~1. Use the same features that computed above for the test set, except the distance based features.~~

<a id='sec2.2'></a>

## 2.2 Training DataFrame

Training data are generated as follows:
1. each input tuple $(\text{startPOI}, \text{endPOI}, \text{#POI})$ form a `query` (in IR terminology).
1. the label of a specific POI is the number of presence of that POI in a specific `query`, excluding the presence as $\text{startPOI}$ or $\text{endPOI}$.
1. for each `query`, the label of all absence POIs from trajectories of that `query` in training set got a label 0.

The dimension of training data matrix is `#(qid, poi)` by `#feature`.

In [ ]:
def gen_train_subdf(poi_id, query_id_set, columns, poi_info, query_id_rdict):
    df_ = pd.DataFrame(data=np.zeros((len(query_id_set), len(columns)), dtype=np.float), columns=columns)
    
    lon = poi_info.loc[poi_id, 'poiLon']; lat = poi_info.loc[poi_id, 'poiLat']
    pop = poi_info.loc[poi_id, 'popularity']; nvisit = poi_info.loc[poi_id, 'nVisit']
    cat = poi_info.loc[poi_id, 'poiCat']; duration = poi_info.loc[poi_id, 'avgDuration']
    
    for j in range(len(query_id_set)):
        qid = query_id_set[j]
        assert(qid in query_id_rdict) # qid --> (start, end, length)
        (p0, pN, trajLen) = query_id_rdict[qid]
        lon0 = poi_info.loc[p0, 'poiLon']; lat0 = poi_info.loc[p0, 'poiLat']
        lonN = poi_info.loc[pN, 'poiLon']; latN = poi_info.loc[pN, 'poiLat']
        idx = df_.index[j]
        df_.loc[idx, 'poiID'] = poi_id
        df_.loc[idx, 'queryID'] = qid
        df_.loc[idx, 'popularity'] = pop
        df_.loc[idx, 'nVisit'] = nvisit
        df_.loc[idx, 'avgDuration'] = duration
        df_.loc[idx, 'sameCatStart'] = 1 if cat == poi_info.loc[p0, 'poiCat'] else -1
        df_.loc[idx, 'sameCatEnd']   = 1 if cat == poi_info.loc[pN, 'poiCat'] else -1
        df_.loc[idx, 'distStart'] = 0 if poi_id == p0 else calc_dist(lon, lat, lon0, lat0)
        df_.loc[idx, 'distEnd']   = 0 if poi_id == pN else calc_dist(lon, lat, lonN, latN)
        df_.loc[idx, 'trajLen'] = trajLen
        df_.loc[idx, 'diffPopStart'] = 0 if poi_id == p0 else pop - poi_info.loc[p0, 'popularity']
        df_.loc[idx, 'diffPopEnd']   = 0 if poi_id == pN else pop - poi_info.loc[pN, 'popularity']
        df_.loc[idx, 'diffNVisitStart'] = 0 if poi_id == p0 else nvisit - poi_info.loc[p0, 'nVisit']
        df_.loc[idx, 'diffNVisitEnd']   = 0 if poi_id == pN else nvisit - poi_info.loc[pN, 'nVisit']
        df_.loc[idx, 'diffDurationStart'] = 0 if poi_id == p0 else duration - poi_info.loc[p0, 'avgDuration']
        df_.loc[idx, 'diffDurationEnd']   = 0 if poi_id == pN else duration - poi_info.loc[pN, 'avgDuration']
        
    return df_

In [ ]:
def gen_train_df(trajid_list, columns, traj_all, poi_info, query_id_dict):
    columns = columns.copy()
    train_trajs = [extract_traj(tid, traj_all) for tid in trajid_list]
    
    qid_set = sorted(set([query_id_dict[(t[0], t[-1], len(t))] for t in train_trajs if len(t) > 2]))
    poi_set = sorted(set(poi_info.index.tolist()))
    
    #qid_poi_pair = list(itertools.product(qid_set, poi_set)) # Cartesian product of qid_set and poi_set
    #df_ = pd.DataFrame(data=np.zeros((len(qid_poi_pair), len(columns)), dtype= np.float), columns=columns)
    
    query_id_rdict = dict()
    for k, v in query_id_dict.items(): 
        query_id_rdict[v] = k  # qid --> (start, end, length)
    
    train_df_list = Parallel(n_jobs=-2)\
                            (delayed(gen_train_subdf)(poi, qid_set, columns, poi_info, query_id_rdict) \
                             for poi in poi_set)
                        
    assert(len(train_df_list) > 0)
    df_ = train_df_list[0]
    for j in range(1, len(train_df_list)):
        df_ = df_.append(train_df_list[j], ignore_index=True)            
        
    # set label
    df_.set_index(['queryID', 'poiID'], inplace=True)
    for t in train_trajs:
        if len(t) > 2:
            qid = query_id_dict[(t[0], t[-1], len(t))]
            for poi in t[1:-1]:  # do NOT count if the POI is startPOI/endPOI
                df_.loc[(qid, poi), 'label'] += 1
    
    df_.reset_index(inplace=True)
    return df_

Sanity check: 
- different POIs have different features for the same query trajectory
- the same POI get different features for different query-id

<a id='sec2.3'></a>

## 2.3 Test DataFrame

Test data are generated the same way as training data, except that the labels of testing data (unknown) could be arbitrary values as suggested in [libsvm FAQ](http://www.csie.ntu.edu.tw/~cjlin/libsvm/faq.html#f431).
The reported accuracy (by `svm-predict` command) is meaningless as it is calculated based on these labels.

The dimension of training data matrix is `#poi` by `#feature` with one specific `query`, i.e. tuple $(\text{startPOI}, \text{endPOI}, \text{#POI})$.

In [ ]:
def gen_test_df(startPOI, endPOI, nPOI, columns, poi_info, query_id_dict):
    columns = columns.copy()
    key = (p0, pN, trajLen) = (startPOI, endPOI, nPOI)
    assert(key in query_id_dict)
    
    poi_set = sorted(set(poi_info.index.tolist()))
    df_ = pd.DataFrame(data=np.zeros((len(poi_set), len(columns)), dtype= np.float), columns=columns)
    
    qid = query_id_dict[key]
    df_['queryID'] = qid
    df_['label'] = np.random.rand(df_.shape[0]) # label for test data is arbitrary according to libsvm FAQ

    lon0 = poi_info.loc[p0, 'poiLon']; lat0 = poi_info.loc[p0, 'poiLat']
    lonN = poi_info.loc[pN, 'poiLon']; latN = poi_info.loc[pN, 'poiLat']
    for i in range(df_.index.shape[0]):
        poi = poi_set[i]
        lon = poi_info.loc[poi, 'poiLon']; lat = poi_info.loc[poi, 'poiLat']
        pop = poi_info.loc[poi, 'popularity']; nvisit = poi_info.loc[poi, 'nVisit']
        cat = poi_info.loc[poi, 'poiCat']; duration = poi_info.loc[poi, 'avgDuration']
        idx = df_.index[i]
        df_.loc[idx, 'poiID'] = poi 
        df_.loc[idx, 'popularity'] = pop
        df_.loc[idx, 'nVisit'] = nvisit
        df_.loc[idx, 'avgDuration'] = duration
        df_.loc[idx, 'sameCatStart'] = 1 if cat == poi_info.loc[p0, 'poiCat'] else -1
        df_.loc[idx, 'sameCatEnd']   = 1 if cat == poi_info.loc[pN, 'poiCat'] else -1
        df_.loc[idx, 'distStart'] = 0 if poi == p0 else calc_dist(lon, lat, lon0, lat0)
        df_.loc[idx, 'distEnd']   = 0 if poi == pN else calc_dist(lon, lat, lonN, latN)
        df_.loc[idx, 'trajLen'] = trajLen
        df_.loc[idx, 'diffPopStart'] = 0 if poi == p0 else pop - poi_info.loc[p0, 'popularity']
        df_.loc[idx, 'diffPopEnd']   = 0 if poi == pN else pop - poi_info.loc[pN, 'popularity']
        df_.loc[idx, 'diffNVisitStart'] = 0 if poi == p0 else nvisit - poi_info.loc[p0, 'nVisit']
        df_.loc[idx, 'diffNVisitEnd']   = 0 if poi == pN else nvisit - poi_info.loc[pN, 'nVisit']
        df_.loc[idx, 'diffDurationStart'] = 0 if poi == p0 else duration - poi_info.loc[p0, 'avgDuration']
        df_.loc[idx, 'diffDurationEnd']   = 0 if poi == pN else duration - poi_info.loc[pN, 'avgDuration']
    return df_

Sanity check: 
- different POIs have different features for the same query trajectory
- the same POI get different features for different query-id

Generate a string for a training/test data frame.

In [ ]:
def gen_data_str(df_):
    columns = df_columns[1:].copy()  # get rid of 'poiID'
    for col in columns:
        assert(col in df_.columns)
        
    lines = []
    for idx in df_.index:
        slist = [str(df_.loc[idx, 'label'])]
        slist.append(' qid:')
        slist.append(str(int(df_.loc[idx, 'queryID'])))
        for j in range(2, len(columns)):
            slist.append(' ')
            slist.append(str(j-1))
            slist.append(':')
            slist.append(str(df_.loc[idx, columns[j]]))
        slist.append('\n')
        lines.append(''.join(slist))
    return ''.join(lines)

<a id='sec2.4'></a>

## 2.4 Ranking POIs using rankSVM

RankSVM implementation in [libsvm.zip](https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/ranksvm/libsvm-ranksvm-3.20.zip) or [liblinear.zip](https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/ranksvm/liblinear-ranksvm-1.95.zip), please read `README.ranksvm` in the zip file for installation instructions.

Use [softmax function](https://en.wikipedia.org/wiki/Softmax_function) to convert ranking scores to a probability distribution.

In [ ]:
def softmax(x):
    expx = np.exp(x)
    return expx / np.sum(expx, axis=0) # column-wise sum

Below is a python wrapper of the `svm-train` or `train` and `svm-predict` or `predict` commands of rankSVM with ranking probabilities $P(p_i \lvert (p_s, p_e, len))$ computed using [softmax function](https://en.wikipedia.org/wiki/Softmax_function).

In [ ]:
# python wrapper of rankSVM
class RankSVM:
    def __init__(self, bin_dir, useLinear=True, debug=False):
        dir_ = !echo $bin_dir  # deal with environmental variables in path
        assert(os.path.exists(dir_[0]))
        self.bin_dir = dir_[0]
        
        self.bin_train = 'svm-train'
        self.bin_predict = 'svm-predict'
        if useLinear:
            self.bin_train = 'train'
            self.bin_predict = 'predict'
        
        assert(isinstance(debug, bool))
        self.debug = debug
        
        # create named tmp files for model and feature scaling parameters
        self.fmodel = None
        self.fscale = None
        with tempfile.NamedTemporaryFile(delete=False) as fd: 
            self.fmodel = fd.name
        with tempfile.NamedTemporaryFile(delete=False) as fd: 
            self.fscale = fd.name
        
        if self.debug:
            print('model file:', self.fmodel)
            print('feature scaling parameter file:', self.fscale)
    
    
    def __del__(self):
        # remove tmp files
        if self.fmodel is not None and os.path.exists(self.fmodel):
            os.unlink(self.fmodel)
        if self.fscale is not None and os.path.exists(self.fscale):
            os.unlink(self.fscale)
    
    
    def train(self, train_df, cost=1):
        # cost is parameter C in SVM
        # write train data to file
        ftrain = None
        with tempfile.NamedTemporaryFile(mode='w+t', delete=False) as fd: 
            ftrain = fd.name
            datastr = gen_data_str(train_df)
            fd.write(datastr)
        
        # feature scaling
        ftrain_scaled = None
        with tempfile.NamedTemporaryFile(mode='w+t', delete=False) as fd: 
            ftrain_scaled = fd.name
        result = !$self.bin_dir/svm-scale -s $self.fscale $ftrain > $ftrain_scaled
        
        if self.debug:
            print('cost:', cost)
            print('train data file:', ftrain)
            print('feature scaled train data file:', ftrain_scaled)
        
        # train rank svm and generate model file, if the model file exists, rewrite it
        #n_cv = 10  # parameter k for k-fold cross-validation, NO model file will be generated in CV mode
        #result = !$self.bin_dir/svm-train -c $cost -v $n_cv $ftrain $self.fmodel
        result = !$self.bin_dir/$self.bin_train -c $cost $ftrain_scaled $self.fmodel
        if self.debug:
            print('Training finished.')
            for i in range(len(result)): print(result[i])

        # remove train data file
        os.unlink(ftrain)
        os.unlink(ftrain_scaled)        
    
    
    def predict(self, test_df):
        # predict ranking scores for the given feature matrix
        if self.fmodel is None or not os.path.exists(self.fmodel):
            print('Model should be trained before predicting')
            return
        
        # write test data to file
        ftest = None
        with tempfile.NamedTemporaryFile(mode='w+t', delete=False) as fd: 
            ftest = fd.name
            datastr = gen_data_str(test_df)
            fd.write(datastr)
                
        # feature scaling
        ftest_scaled = None
        with tempfile.NamedTemporaryFile(delete=False) as fd: 
            ftest_scaled = fd.name
        result = !$self.bin_dir/svm-scale -r $self.fscale $ftest > $ftest_scaled
            
        # generate prediction file
        fpredict = None
        with tempfile.NamedTemporaryFile(delete=False) as fd: 
            fpredict = fd.name
            
        if self.debug:
            print('test data file:', ftest)
            print('feature scaled test data file:', ftest_scaled)
            print('predict result file:', fpredict)
            
        # predict using trained model and write prediction to file
        result = !$self.bin_dir/$self.bin_predict $ftest_scaled $self.fmodel $fpredict
        if self.debug:
            print('Predict result: %-30s  %s' % (result[0], result[1]))
        
        # generate prediction DataFrame from prediction file
        poi_rank_df = pd.read_csv(fpredict, header=None)
        poi_rank_df.rename(columns={0:'rank'}, inplace=True)
        poi_rank_df['poiID'] = test_df['poiID'].astype(np.int)
        #poi_rank_df.set_index('poiID', inplace=True) # duplicated 'poiID' when evaluating training data
        poi_rank_df['probability'] = softmax(poi_rank_df['rank'])  # softmax
        
        # remove test file and prediction file
        os.unlink(ftest)
        os.unlink(ftest_scaled)
        os.unlink(fpredict)
        
        return poi_rank_df

Sanity check.

In [ ]:
poi_info_ = calc_poi_info(trajid_set_all, traj_all, poi_all)

In [ ]:
train_df_ = gen_train_df(trajid_set_all, df_columns, traj_all, poi_info_, query_id_dict)

In [ ]:
ranksvm_ = RankSVM(ranksvm_dir, useLinear=True)
ranksvm_.train(train_df_, cost=1000)

Histograms on training set.

In [ ]:
rank_train_ = ranksvm_.predict(train_df_)

In [ ]:
print(train_df_.shape, rank_train_.shape)

In [ ]:
labels = sorted(train_df_['label'].unique())
labels

In [ ]:
rmin_ = rank_train_['rank'].min()
rmax_ = rank_train_['rank'].max()
print(rmin_, rmax_)
rmin1_ = np.round(rmin_); rmin1_ = rmin1_ if rmin1_ < rmin_ else rmin1_ - 1
rmax1_ = np.round(rmax_); rmax1_ = rmax1_ if rmax1_ > rmax_ else rmax1_ + 1
print(rmin1_, rmax1_)

In [ ]:
pmin_ = rank_train_['probability'].min()
pmax_ = rank_train_['probability'].max()
print(pmin_, pmax_)
pmin1_ = np.round(pmin_); pmin1_ = pmin1_ if pmin1_ < pmin_ else max(0, pmin1_ - 0.001)
pmax1_ = np.round(pmax_); pmax1_ = pmax1_ if pmax1_ > pmax_ else min(1, pmax1_ + 0.001)
print(pmin1_, pmax1_)

In [ ]:
plt.figure(figsize=[15, 4])
ax1 = plt.subplot(1, 3, 1); plt.title('Histogram of Lables (Train)')
train_df_['label'].hist(ax=ax1, bins=2*len(labels))
ax1.set_yscale('log')

ax2 = plt.subplot(1, 3, 2); plt.title('Histogram of Ranking Score (Train)')
#hist1, bins1 = np.histogram(rank_train_['rank'], bins=5)
bins1_train = np.r_[np.arange(rmin1_, rmax1_, 0.5), rmax1_]  # [-1.5, -1.0, -0.5, 0, 0.5, 1.0, 1.5, 2.0]
for l in labels:
    ix = train_df_[train_df_['label'] == l].index
    rank_train_.loc[ix, 'rank'].hist(ax=ax2, bins=bins1_train, label='Label=%d' % l, alpha=0.7)
ax2.set_yscale('log')
ax2.set_ylim(ymin=0.1)
ax2.set_xlabel('Ranking Score')
plt.legend(loc='upper left')

ax3 = plt.subplot(1, 3, 3); plt.title('Histogram of Ranking Probability (Train)')
#hist2, bins2 = np.histogram(rank_train_['probability'], bins=5)
bins2_train = np.r_[np.arange(pmin1_, pmax1_, 0.0001), pmax1_]  # [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45]
for l in labels:
    ix = train_df_[train_df_['label'] == l].index
    rank_train_.loc[ix, 'probability'].hist(ax=ax3, bins=bins2_train, label='Label=%d' % l, alpha=0.7)
ax3.set_yscale('log')
ax3.set_ylim(ymin=0.1)
ax3.set_xlabel('Ranking Probability')
plt.legend(loc='upper right')
print('NOTE: labels of POI in training set is query specific')

<a id='sec2.5'></a>

## 2.5 Recommendation based on POI Ranking

Recommend trajectories based on ranking of POIs.

In [ ]:
if run_rank == True:
    recdict_rank = dict()
    cost = 1000
    cnt = 0
    for i in range(len(trajid_set_all)):
        tid = trajid_set_all[i]
        t = extract_traj(tid, traj_all)
        
        # trajectory too short
        if len(t) < 3: continue
            
        # TSP, generally an ILP very hard for solvers (GUROBI/CBC)
        if t[0] == t[-1]: continue

        if uspecific == True:
            user = traj_user.loc[tid, 'userID']
            trajid_set_user = traj_user[traj_user['userID'] == user].index.tolist()
            trajid_set_other = traj_user[traj_user['userID'] != user].index.tolist()
            assert(tid in trajid_set_user)
            trajid_set_add = [x for x in trajid_set_user if x != tid]
            trajid_list_train = trajid_set_other.copy()
            assert(KX > 0)
            for k in range(KX):
                trajid_list_train = trajid_list_train + trajid_set_add
        else:
            trajid_list_train = trajid_set_all[:i] + trajid_set_all[i+1:]

        poi_info = calc_poi_info(trajid_list_train, traj_all, poi_all)
        
        # trajectory too long
        if len(t) > poi_info.shape[0]: continue
        
        print(t, '#%d ->' % cnt); cnt += 1; sys.stdout.flush()

        # POI popularity based ranking
        poi_info.sort_values(by='popularity', ascending=False, inplace=True)
        ranks1 = poi_info.index.tolist()
        rec_pop = [t[0]] + [x for x in ranks1 if x not in {t[0], t[-1]}][:len(t)-2] + [t[-1]]

        # POI feature based ranking
        train_df = gen_train_df(trajid_list_train, df_columns, traj_all, poi_info, query_id_dict)
        ranksvm = RankSVM(ranksvm_dir, useLinear=True)
        ranksvm.train(train_df, cost=cost)
        test_df = gen_test_df(t[0], t[-1], len(t), df_columns, poi_info, query_id_dict)
        rank_df = ranksvm.predict(test_df)
        rank_df.sort_values(by='rank', ascending=False, inplace=True)
        ranks2 = rank_df['poiID'].tolist()
        rec_feature = [t[0]] + [x for x in ranks2 if x not in {t[0], t[-1]}][:len(t)-2] + [t[-1]]

        recdict_rank[tid] = {'REAL':t, 'REC_POP':rec_pop, 'REC_FEATURE':rec_feature}

        print(' '*10, rec_pop)
        print(' '*10, rec_feature); sys.stdout.flush()

In [ ]:
if run_rank == True:
    pickle.dump(recdict_rank, open(frecdict_rank, 'bw'))

In [ ]:
if run_rank == True:
    F11_rank = []
    F12_rank = []
    for key in sorted(recdict_rank.keys()):
        F11_rank.append(calc_F1(recdict_rank[key]['REAL'], recdict_rank[key]['REC_POP']))
        F12_rank.append(calc_F1(recdict_rank[key]['REAL'], recdict_rank[key]['REC_FEATURE']))
    print('POP    : %.3f, %.3f' % (np.mean(F11_rank), np.std(F11_rank)))
    print('FEATURE: %.3f, %.3f' % (np.mean(F12_rank), np.std(F12_rank)))

In [ ]:
# Glas
# user agnostic
#POP    : 0.706, 0.169
#FEATURE: 0.758, 0.182
#
# user specific
#POP    : 0.706, 0.169
#FEATURE: 0.754, 0.179

# Osak
# user agnostic
#POP    : 0.611, 0.163
#FEATURE: 0.633, 0.153
#
# user specific
#POP    : 0.611, 0.163
#FEATURE: 0.647, 0.170

<a id='sec3'></a>

# 3. Factorise Transition Probabilities in POI Feature Space

Estimate a transition matrix for each feature of POI, transition probabilities (matrix) between different POI features (vector) is obtrained by the [Kronecker product](https://en.wikipedia.org/wiki/Kronecker_product) of the individual transition matrix corresponding to each feature.

<a id='sec3.1'></a>

## 3.1 POI Features for Factorisation

POI features used to factorise transition matrix of Markov Chain with POI features (vector) as states:
- Category of POI
- Popularity of POI (discritize with uniform log-scale bins, #bins <=5 )
- The number of POI visits (discritize with uniform log-scale bins, #bins <=5 )
- The average visit duration of POI (discritise with uniform log-scale bins, #bins <= 5)
- The neighborhood relationship between POIs (clustering POI(lat, lon) using k-means, #clusters <= 5)

We count the number of transition first, then normalise each row while taking care of zero by adding each cell a small number (i.e. $0.2$ times the minimum value of that row) if there exists a zero cell.

In [ ]:
def normalise_transmat(transmat):
    assert(isinstance(transmat, pd.DataFrame))
    for row in range(transmat.index.shape[0]):
        nonzeroidx = np.nonzero(transmat.iloc[row])[0].tolist()    
        if len(nonzeroidx) < transmat.columns.shape[0]:
            if len(nonzeroidx) == 0:  # all zeros in row
                transmat.iloc[row] = 1 / transmat.columns.shape[0]  # uniform distribution
            else:
                minv = np.min(transmat.iloc[row, nonzeroidx])
                EPS = 0.2 * minv  # row-specific smooth factor
                #zeroidx = list(set(range(len(transmat.columns))) - set(nonzeroidx))
                #transmat.iloc[row, zeroidx] = EPS
                transmat.iloc[row] += EPS
                rowsum = np.sum(transmat.iloc[row])
                assert(rowsum > 0)
                transmat.iloc[row] /= rowsum
        else:
            assert(len(nonzeroidx) == transmat.columns.shape[0])  # all non-zero in row
            transmat.iloc[row] /= np.sum(transmat.iloc[row])
    return transmat

POIs in training set.

In [ ]:
poi_train_ = traj_all[traj_all['trajID'].isin(trajid_set_all)]['poiID'].unique().tolist()
poi_train_.sort()
#poi_train

<a id='sec3.2'></a>

## 3.2 Transition Matrix between POI Cateogries

In [ ]:
poi_cats = poi_all.loc[poi_train_, 'poiCat'].unique().tolist()
poi_cats.sort()
poi_cats

In [ ]:
transmat_cat = pd.DataFrame(data=np.zeros((len(poi_cats), len(poi_cats)), dtype=np.float), \
                            columns=poi_cats, index=poi_cats)

Count the number of transitions between POI categories.

In [ ]:
for tid in trajid_set_all:
    t = extract_traj(tid, traj_all)
    if len(t) > 1:
        for pi in range(len(t)-1):
            p1 = t[pi]
            p2 = t[pi+1]
            cat1 = poi_all.loc[p1, 'poiCat']
            cat2 = poi_all.loc[p2, 'poiCat']
            transmat_cat.loc[cat1, cat2] += 1

In [ ]:
transmat_cat

Normalise each row to obtain transition probabilities.

In [ ]:
transmat_cat = normalise_transmat(transmat_cat)

In [ ]:
transmat_cat

Combine the above steps.

In [ ]:
def gen_transmat_cat(trajid_list, traj_all, poi_info, poi_cats=poi_cats):
    #poi_train = poi_info.index.tolist()
    #poi_cats = poi_info.loc[poi_train, 'poiCat'].unique().tolist()
    #poi_cats.sort()
    
    transmat_cat = pd.DataFrame(data=np.zeros((len(poi_cats), len(poi_cats)), dtype=np.float), \
                                columns=poi_cats, index=poi_cats)
    for tid in trajid_list:
        t = extract_traj(tid, traj_all)
        if len(t) > 1:
            for pi in range(len(t)-1):
                p1 = t[pi]
                p2 = t[pi+1]
                cat1 = poi_info.loc[p1, 'poiCat']
                cat2 = poi_info.loc[p2, 'poiCat']
                transmat_cat.loc[cat1, cat2] += 1
    return normalise_transmat(transmat_cat)

<a id='sec3.3'></a>

## 3.3 Transition Matrix between POI Popularity Classes

In [ ]:
poi_pops = poi_info_.loc[poi_train_, 'popularity']
sorted(poi_pops.unique().tolist())

Discretize POI popularity with uniform log-scale bins (#bins $\le 5$).

In [ ]:
expo_pop1 = np.log10(max(1, min(poi_pops)))
expo_pop2 = np.log10(max(poi_pops))
print(expo_pop1, expo_pop2)

In [ ]:
nbins_pop = 2
logbins_pop = np.logspace(np.floor(expo_pop1), np.ceil(expo_pop2), nbins_pop+1)
if min(poi_pops) == 0: 
    logbins_pop[0] = 0
if uspecific == True:
    logbins_pop[-1] = KX * logbins_pop[-1]  # deal with overflow
logbins_pop

In [ ]:
ax = pd.Series(poi_pops).hist(figsize=(5, 3), bins=logbins_pop)
ax.set_xscale('log')

In [ ]:
transmat_pop = pd.DataFrame(data=np.zeros((nbins_pop, nbins_pop), dtype=np.float), \
                            columns=np.arange(1, nbins_pop+1), index=np.arange(1, nbins_pop+1))

Count the number of transitions between POI popularity classes.

In [ ]:
for tid in trajid_set_all:
    t = extract_traj(tid, traj_all)
    if len(t) > 1:
        for pi in range(len(t)-1):
            p1 = t[pi]
            p2 = t[pi+1]
            pop1 = poi_info_.loc[p1, 'popularity']
            pop2 = poi_info_.loc[p2, 'popularity']
            pc1, pc2 = np.digitize([pop1, pop2], logbins_pop)
            transmat_pop.loc[pc1, pc2] += 1

In [ ]:
transmat_pop

Normalise each row to obtain transition probabilities.

In [ ]:
transmat_pop = normalise_transmat(transmat_pop)

In [ ]:
transmat_pop

Combine the above steps.

In [ ]:
def gen_transmat_pop(trajid_list, traj_all, poi_info, bins=logbins_pop):
    #poi_train = poi_info.index.tolist()
    #poi_pops = poi_info.loc[poi_train, 'popularity'].unique().tolist()
    #poi_pops.sort()
    #expo_pop1 = np.log10(min(poi_pops))
    #expo_pop2 = np.log10(max(poi_pops))
    #logbins_pop = np.logspace(np.floor(expo_pop1), np.ceil(expo_pop2), nbins+1)
    
    nbins = len(bins) - 1
    transmat_pop = pd.DataFrame(data=np.zeros((nbins, nbins), dtype=np.float), \
                                columns=np.arange(1, nbins+1), index=np.arange(1, nbins+1))
    for tid in trajid_list:
        t = extract_traj(tid, traj_all)
        if len(t) > 1:
            for pi in range(len(t)-1):
                p1 = t[pi]
                p2 = t[pi+1]
                pop1 = poi_info.loc[p1, 'popularity']
                pop2 = poi_info.loc[p2, 'popularity']
                pc1, pc2 = np.digitize([pop1, pop2], logbins_pop)
                transmat_pop.loc[pc1, pc2] += 1
    return normalise_transmat(transmat_pop), logbins_pop

<a id='sec3.4'></a>

## 3.4 Transition Matrix between the Number of POI Visit Classes

In [ ]:
poi_visits = poi_info_.loc[poi_train_, 'nVisit']
sorted(poi_visits.unique().tolist())

Discretize the number of POI visit with uniform log-scale bins (#bins $\le 5$).

In [ ]:
expo_visit1 = np.log10(max(1, min(poi_visits)))
expo_visit2 = np.log10(max(poi_visits))
print(expo_visit1, expo_visit2)

In [ ]:
nbins_visit = 2
logbins_visit = np.logspace(np.floor(expo_visit1), np.ceil(expo_visit2), nbins_visit+1)
if min(poi_visits) == 0: 
    logbins_visit[0] = 0
if uspecific == True:
    logbins_visit[-1] = KX * logbins_visit[-1]  # deal with overflow
logbins_visit

In [ ]:
ax = pd.Series(poi_visits).hist(figsize=(5, 3), bins=logbins_visit)
ax.set_xscale('log')

In [ ]:
transmat_visit = pd.DataFrame(data=np.zeros((nbins_visit, nbins_visit), dtype=np.float), \
                              columns=np.arange(1, nbins_visit+1), index=np.arange(1, nbins_visit+1))

Count the number of transitions between POI popularity classes.

In [ ]:
for tid in trajid_set_all:
    t = extract_traj(tid, traj_all)
    if len(t) > 1:
        for pi in range(len(t)-1):
            p1 = t[pi]
            p2 = t[pi+1]
            visit1 = poi_info_.loc[p1, 'nVisit']
            visit2 = poi_info_.loc[p2, 'nVisit']
            vc1, vc2 = np.digitize([visit1, visit2], logbins_visit)
            transmat_visit.loc[vc1, vc2] += 1

In [ ]:
transmat_visit

Normalise each row to obtain transition probabilities.

In [ ]:
transmat_visit = normalise_transmat(transmat_visit)

In [ ]:
transmat_visit

Combine the above steps.

In [ ]:
def gen_transmat_visit(trajid_list, traj_all, poi_info, bins=logbins_visit):
    #poi_train = poi_info.index.tolist()
    #poi_visits = poi_info.loc[poi_train, 'nVisit'].unique().tolist()
    #poi_visits.sort()
    #expo_visit1 = np.log10(min(poi_visits))
    #expo_visit2 = np.log10(max(poi_visits))
    #logbins_visit = np.logspace(np.floor(expo_visit1), np.ceil(expo_visit2), nbins+1)
    
    nbins = len(logbins_visit) - 1
    transmat_visit = pd.DataFrame(data=np.zeros((nbins, nbins), dtype=np.float), \
                                  columns=np.arange(1, nbins+1), index=np.arange(1, nbins+1))
    for tid in trajid_list:
        t = extract_traj(tid, traj_all)
        if len(t) > 1:
            for pi in range(len(t)-1):
                p1 = t[pi]
                p2 = t[pi+1]
                visit1 = poi_info.loc[p1, 'nVisit']
                visit2 = poi_info.loc[p2, 'nVisit']
                vc1, vc2 = np.digitize([visit1, visit2], logbins_visit)
                transmat_visit.loc[vc1, vc2] += 1
    return normalise_transmat(transmat_visit), logbins_visit

<a id='sec3.5'></a>

## 3.5 Transition Matrix between POI Average Visit Duration Classes

In [ ]:
poi_durations = poi_info_.loc[poi_train_, 'avgDuration']
sorted(poi_durations.unique().tolist())

Discretize POI average visit duration with uniform log-scale bins (#bins $\le 5$).

In [ ]:
expo_duration1 = np.log10(max(1, min(poi_durations)))
expo_duration2 = np.log10(max(poi_durations))
print(expo_duration1, expo_duration2)

In [ ]:
nbins_duration = 3
logbins_duration = np.logspace(np.floor(expo_duration1), np.ceil(expo_duration2), nbins_duration+1)
if min(poi_durations) == 0: 
    logbins_duration[0] = 0
if uspecific == True:
    logbins_duration[-1] = KX * logbins_duration[-1]  # deal with overflow
logbins_duration

In [ ]:
ax = pd.Series(poi_durations).hist(figsize=(5, 3), bins=logbins_duration)
ax.set_xscale('log')

In [ ]:
transmat_duration = pd.DataFrame(data=np.zeros((nbins_duration, nbins_duration), dtype=np.float), \
                                 columns=np.arange(1, nbins_duration+1), index=np.arange(1, nbins_duration+1))

Count the number of transitions between POI average visit duration classes.

In [ ]:
for tid in trajid_set_all:
    t = extract_traj(tid, traj_all)
    if len(t) > 1:
        for pi in range(len(t)-1):
            p1 = t[pi]
            p2 = t[pi+1]
            d1 = poi_info_.loc[p1, 'avgDuration']
            d2 = poi_info_.loc[p2, 'avgDuration']
            dc1, dc2 = np.digitize([d1, d2], logbins_duration)
            transmat_duration.loc[dc1, dc2] += 1

In [ ]:
transmat_duration

Normalise each row to obtain transition probabilities.

In [ ]:
transmat_duration = normalise_transmat(transmat_duration)

In [ ]:
transmat_duration

Combine the above steps.

In [ ]:
def gen_transmat_duration(trajid_list, traj_all, poi_info, bins=logbins_duration):
    #poi_train = poi_info.index.tolist()
    #poi_durations = poi_info.loc[poi_train, 'avgDuration'].unique().tolist()
    #poi_durations.sort()
    #expo_duration1 = np.log10(min(poi_durations))
    #expo_duration2 = np.log10(max(poi_durations))
    #logbins_duration = np.logspace(np.floor(expo_duration1), np.ceil(expo_duration2), nbins+1)
    
    nbins = len(logbins_duration) - 1
    transmat_duration = pd.DataFrame(data=np.zeros((nbins, nbins), dtype=np.float), \
                                     columns=np.arange(1, nbins+1), index=np.arange(1, nbins+1))
    for tid in trajid_list:
        t = extract_traj(tid, traj_all)
        if len(t) > 1:
            for pi in range(len(t)-1):
                p1 = t[pi]
                p2 = t[pi+1]
                d1 = poi_info.loc[p1, 'avgDuration']
                d2 = poi_info.loc[p2, 'avgDuration']
                dc1, dc2 = np.digitize([d1, d2], logbins_duration)
                transmat_duration.loc[dc1, dc2] += 1
    return normalise_transmat(transmat_duration), logbins_duration

<a id='sec3.6'></a>

## 3.6 Transition Matrix between POI Neighborhood Classes

Compute POI neighborhoods, i.e., clustering POI (lat, lon) in training set using k-means, #clusters $\le 5$.

KMeans in scikit-learn seems unable to use custom distance metric and no implementation of [Haversine formula](http://en.wikipedia.org/wiki/Great-circle_distance), use Euclidean distance to approximate.

In [ ]:
#from sklearn.cluster import KMeans
X = poi_all.loc[poi_train_, ['poiLon', 'poiLat']]
nclusters = 4

In [ ]:
kmeans = KMeans(n_clusters=nclusters)
kmeans.fit(X)

In [ ]:
clusters = kmeans.predict(X)
clusters

In [ ]:
poi_clusters = pd.DataFrame(data=clusters, index=poi_train_)
poi_clusters.index.name = 'poiID'
poi_clusters.rename(columns={0:'clusterID'}, inplace=True)
#poi_clusters

In [ ]:
diff = poi_all.loc[poi_train_, ['poiLon', 'poiLat']].max() - poi_all.loc[poi_train_, ['poiLon', 'poiLat']].min()
ratio = diff['poiLon'] / diff['poiLat']
ratio

Scatter plot of POI coordinates with clustering results.

In [ ]:
height = 4; width = int(round(ratio)*height)
plt.figure(figsize=[width, height])
plt.scatter(poi_all.loc[poi_train_, 'poiLon'], poi_all.loc[poi_train_, 'poiLat'], c=clusters, s=50)

In [ ]:
transmat_neighbor = pd.DataFrame(data=np.zeros((nclusters, nclusters), dtype=np.float), \
                                 columns=np.arange(nclusters), index=np.arange(nclusters))

Count the number of transitions between POIs in different clusters.

In [ ]:
for tid in trajid_set_all:
    t = extract_traj(tid, traj_all)
    if len(t) > 1:
        for pi in range(len(t)-1):
            p1 = t[pi]
            p2 = t[pi+1]
            c1 = poi_clusters.loc[p1, 'clusterID']
            c2 = poi_clusters.loc[p2, 'clusterID']
            transmat_neighbor.loc[c1, c2] += 1

In [ ]:
transmat_neighbor

Normalise each row to obtain transition probabilities.

In [ ]:
transmat_neighbor = normalise_transmat(transmat_neighbor)

In [ ]:
transmat_neighbor

Combine the above steps.

In [ ]:
def gen_transmat_neighbor(trajid_list, traj_all, poi_info, poi_clusters=poi_clusters):
    #poi_train = poi_info.index.tolist()
    #X = poi_info.loc[poi_train, ['poiLon', 'poiLat']]
    #kmeans = KMeans(n_clusters=nclusters)
    #kmeans.fit(X)
    #clusters = kmeans.predict(X)
    #poi_clusters = pd.DataFrame(data=clusters, index=poi_train)
    #poi_clusters.index.name = 'poiID'
    #poi_clusters.rename(columns={0:'clusterID'}, inplace=True)
    
    nclusters = len(poi_clusters['clusterID'].unique())
    transmat_neighbor = pd.DataFrame(data=np.zeros((nclusters, nclusters), dtype=np.float), \
                                     columns=np.arange(nclusters), index=np.arange(nclusters))
    for tid in trajid_list:
        t = extract_traj(tid, traj_all)
        if len(t) > 1:
            for pi in range(len(t)-1):
                p1 = t[pi]
                p2 = t[pi+1]
                c1 = poi_clusters.loc[p1, 'clusterID']
                c2 = poi_clusters.loc[p2, 'clusterID']
                transmat_neighbor.loc[c1, c2] += 1
    return normalise_transmat(transmat_neighbor), poi_clusters

<a id='sec3.7'></a>

## 3.7 Transition Matrix between POIs

Approximate transition probabilities (matrix) between different POI features (vector) using the [Kronecker product](https://en.wikipedia.org/wiki/Kronecker_product) of individual transition matrix corresponding to each feature, i.e., POI category, POI popularity (discritized), POI average visit duration (discritized) and POI neighborhoods (clusters).

In [ ]:
#from scipy.linalg import kron
transmat_value = transmat_cat.values
for transmat in [transmat_pop, transmat_visit, transmat_duration, transmat_neighbor]:
    transmat_value = kron(transmat_value, transmat.values)

Index of Kronecker product.

In [ ]:
transmat_ix = list(itertools.product(transmat_cat.index, transmat_pop.index, transmat_visit.index, \
                                     transmat_duration.index, transmat_neighbor.index))

In [ ]:
transmat_feature = pd.DataFrame(data=transmat_value, index=transmat_ix, columns=transmat_ix)
transmat_feature.index.name = '(poiTheme, popularity, avgDuration, clusterID)'
#transmat_feature.head()

In [ ]:
print(transmat_cat.shape)
print(transmat_pop.shape)
print(transmat_visit.shape)
print(transmat_duration.shape)
print(transmat_neighbor.shape)
print(transmat_feature.shape)

In [ ]:
transmat_feature.head()

The result of [Kronecker product](https://en.wikipedia.org/wiki/Kronecker_product) should be normalised to obtain transition probabilities.

In [ ]:
#from scipy.misc import logsumexp
logtransmat_feature = np.log(transmat_feature)
for row in range(logtransmat_feature.shape[0]):
    logtransmat_feature.iloc[row] -= logsumexp(logtransmat_feature.iloc[row])
logtransmat_feature.head()

In [ ]:
logtransmat_feature.max().sort_values(ascending=False).head(10)

In [ ]:
plt.figure(figsize=[13, 10])
sns.heatmap(np.exp(logtransmat_feature), xticklabels=False, yticklabels=False)

Deal with features without corresponding POIs and feature with more than one corresponding POIs. (*Before Normalisation*)
- For features without corresponding POIs, just remove the rows and columns from the matrix obtained by Kronecker product.
- For different POIs with the exact same feature, 
  - Let POIs with the same feature as a POI group,
  - The *incoming* **transition value (i.e., unnormalised transition probability)** of this POI group 
    should be divided uniformly among the group members, 
    *which corresponds to choose a group member uniformly at random in the incoming case*.
  - The *outgoing* transition value should be duplicated (i.e., the same) among all group members, 
    **as we were already in that group in the outgoing case**.
  - For each POI in the group, the allocation transition value of the *self-loop of the POI group* is similar to 
    that in the *outgoing* case, **as we were already in that group**, so just duplicate and then divide uniformly among 
    the transitions from this POI to other POIs in the same group, 
    *which corresponds to choose a outgoing transition uniformly at random from all outgoing transitions
    excluding the self-loop of this POI*.
- **Concretely**, for a POI group with $n$ POIs, 
    1. If the *incoming* transition value of POI group is $m_1$,
       then the corresponding *incoming* transition value for each group member is $\frac{m_1}{n}$.
    1. If the *outgoing* transition value of POI group is $m_2$,
       then the corresponding *outgoing* transition value for each group member is also $m_2$.
    1. If the transition value of *self-loop of the POI group* is $m_3$,
       then transition value of *self-loop of individual POIs* should be $0$,  
       and *other in-group transitions* with value $\frac{m_3}{n-1}$
       as the total number of outgoing transitions to other POIs in the same group is $n-1$ (excluding the self-loop),
       i.e. $n-1$ choose $1$.
       
**NOTE**: execute the above division before or after row normalisation will lead to the same result, *as the division itself does NOT change the normalising constant of each row (i.e., the sum of each row before normalising)*.

In [ ]:
feature_names = ['poiCat', 'popularity', 'nVisit', 'avgDuration', 'clusterID']
poi_features = pd.DataFrame(data=np.zeros((len(poi_train_), len(feature_names))), \
                            columns=feature_names, index=poi_train_)
poi_features.index.name = 'poiID'

In [ ]:
poi_features['poiCat'] = poi_info_.loc[poi_train_, 'poiCat']
poi_features['popularity'] = np.digitize(poi_info_.loc[poi_train_, 'popularity'], logbins_pop)
poi_features['nVisit'] = np.digitize(poi_info_.loc[poi_train_, 'nVisit'], logbins_visit)
poi_features['avgDuration'] = np.digitize(poi_info_.loc[poi_train_, 'avgDuration'], logbins_duration)
poi_features['clusterID'] = poi_clusters.loc[poi_train_, 'clusterID']

In [ ]:
ix = transmat_feature.index[0]; ix1 = transmat_feature.index[1]
# simply use df.loc[tuple,:] will trigger an error as pandas treat the tuple as multiindex
# see https://github.com/pydata/pandas/issues/7548
#transmat_feature.loc[(ix,),]   # OK
#transmat_feature.loc[(ix,),:]  # OK
transmat_feature.loc[(ix,), (ix1,)] 

In [ ]:
poi_logtransmat = pd.DataFrame(data=np.zeros((len(poi_train_), len(poi_train_)), dtype=np.float), \
                               columns=poi_train_, index=poi_train_)

Copy transition values between features with corresponding POIs.

In [ ]:
for p1 in poi_logtransmat.index:
    rix = tuple(poi_features.loc[p1])
    for p2 in poi_logtransmat.columns:
        cix = tuple(poi_features.loc[p2])
        value_ = transmat_feature.loc[(rix,), (cix,)]
        #print(value_.values)
        poi_logtransmat.loc[p1, p2] = value_.values[0, 0]

In [ ]:
poi_logtransmat.head(11)

Deal with POI features that correspond to more than once POIs.

In [ ]:
features_dup = dict()
for poi in poi_features.index:
    key = tuple(poi_features.loc[poi])
    if key in features_dup:
        features_dup[key].append(poi)
    else:
        features_dup[key] = [poi]
features_dup

In [ ]:
for feature in sorted(features_dup.keys()):
    n = len(features_dup[feature])
    if n > 1:
        group = features_dup[feature]
        v1 = poi_logtransmat.loc[group[0], group[0]]  # transition value of self-loop of POI group
        
        # divide incoming transition value (i.e. unnormalised transition probability) uniformly among group members
        for poi in group:
            poi_logtransmat[poi] /= n
        
        # outgoing transition value has already been duplicated (value copied above)
        
        # duplicate & divide transition value of self-loop of POI group uniformly among all outgoing transitions,
        # from a POI to all other POIs in the same group (excluding POI self-loop)
        v2 = v1 / (n - 1)
        for pair in itertools.permutations(group, 2):
            poi_logtransmat.loc[pair[0], pair[1]] = v2

Make the transition value of all self-loops of POIs to $0$.

In [ ]:
for p1 in poi_logtransmat.index:
    poi_logtransmat.loc[p1, p1] = 0

In [ ]:
poi_logtransmat.head(11)

Normalise and compute log probability.

In [ ]:
for p1 in poi_logtransmat.index:
    rowsum = poi_logtransmat.loc[p1].sum()
    assert(rowsum > 0)
    logrowsum = np.log10(rowsum)
    for p2 in poi_logtransmat.columns:
        if p1 == p2:
            poi_logtransmat.loc[p1, p2] = -np.inf  # deal with log(0) explicitly
        else:
            poi_logtransmat.loc[p1, p2] = np.log10(poi_logtransmat.loc[p1, p2]) - logrowsum

In [ ]:
poi_logtransmat.head(11)

Plot transition matrix heatmap.

In [ ]:
prob_mat = np.power(10, poi_logtransmat)

In [ ]:
plt.figure(figsize=[13, 10])
#plt.imshow(prob_mat, interpolation='none', cmap=plt.cm.hot)  # OK
#ticks = prob_mat.index
#plt.xticks(np.arange(prob_mat.shape[0]), ticks)
#plt.yticks(np.arange(prob_mat.shape[0]), ticks)
#plt.xlabel('POI ID')
#plt.ylabel('POI ID')
sns.heatmap(prob_mat)

In [ ]:
#prob_mat.loc[2, 1]

In [ ]:
#poi_all.loc[[2, 17]]

In [ ]:
#d_ = calc_dist(poi_all.loc[2, 'poiLon'], poi_all.loc[2, 'poiLat'], poi_all.loc[1, 'poiLon'], poi_all.loc[1, 'poiLat'])
#print('Distance between POI %d and %d is %.3f km.' % (2, 1, d_))

Combine the above steps.

In [ ]:
def gen_poi_logtransmat(trajid_list, traj_all, poi_info):
    transmat_cat                        = gen_transmat_cat(trajid_list, traj_all, poi_info)
    transmat_pop,      logbins_pop      = gen_transmat_pop(trajid_list, traj_all, poi_info)
    transmat_visit,    logbins_visit    = gen_transmat_visit(trajid_list, traj_all, poi_info)
    transmat_duration, logbins_duration = gen_transmat_duration(trajid_list, traj_all, poi_info)
    transmat_neighbor, poi_clusters     = gen_transmat_neighbor(trajid_list, traj_all, poi_info)

    # Kronecker product
    transmat_ix = list(itertools.product(transmat_cat.index, transmat_pop.index, transmat_visit.index, \
                                         transmat_duration.index, transmat_neighbor.index))
    transmat_value = transmat_cat.values
    for transmat in [transmat_pop, transmat_visit, transmat_duration, transmat_neighbor]:
        transmat_value = kron(transmat_value, transmat.values)
    transmat_feature = pd.DataFrame(data=transmat_value, index=transmat_ix, columns=transmat_ix)
    
    poi_train = poi_info.index.tolist()
    feature_names = ['poiCat', 'popularity', 'nVisit', 'avgDuration', 'clusterID']
    poi_features = pd.DataFrame(data=np.zeros((len(poi_train), len(feature_names))), \
                                columns=feature_names, index=poi_train)
    poi_features.index.name = 'poiID'
    poi_features['poiCat'] = poi_info.loc[poi_train, 'poiCat']
    poi_features['popularity'] = np.digitize(poi_info.loc[poi_train, 'popularity'], logbins_pop)
    poi_features['nVisit'] = np.digitize(poi_info.loc[poi_train, 'nVisit'], logbins_visit)
    poi_features['avgDuration'] = np.digitize(poi_info.loc[poi_train, 'avgDuration'], logbins_duration)
    poi_features['clusterID'] = poi_clusters.loc[poi_train, 'clusterID']
    
    # shrink the result of Kronecker product and deal with POIs with the same features
    poi_logtransmat = pd.DataFrame(data=np.zeros((len(poi_train), len(poi_train)), dtype=np.float), \
                                   columns=poi_train, index=poi_train)
    for p1 in poi_logtransmat.index:
        rix = tuple(poi_features.loc[p1])
        for p2 in poi_logtransmat.columns:
            cix = tuple(poi_features.loc[p2])
            value_ = transmat_feature.loc[(rix,), (cix,)]
            poi_logtransmat.loc[p1, p2] = value_.values[0, 0]
    
    # group POIs with the same features
    features_dup = dict()
    for poi in poi_features.index:
        key = tuple(poi_features.loc[poi])
        if key in features_dup:
            features_dup[key].append(poi)
        else:
            features_dup[key] = [poi]
            
    # deal with POIs with the same features
    for feature in sorted(features_dup.keys()):
        n = len(features_dup[feature])
        if n > 1:
            group = features_dup[feature]
            v1 = poi_logtransmat.loc[group[0], group[0]]  # transition value of self-loop of POI group
            
            # divide incoming transition value (i.e. unnormalised transition probability) uniformly among group members
            for poi in group:
                poi_logtransmat[poi] /= n
                
            # outgoing transition value has already been duplicated (value copied above)
            
            # duplicate & divide transition value of self-loop of POI group uniformly among all outgoing transitions,
            # from a POI to all other POIs in the same group (excluding POI self-loop)
            v2 = v1 / (n - 1)
            for pair in itertools.permutations(group, 2):
                poi_logtransmat.loc[pair[0], pair[1]] = v2
                            
    # normalise each row
    for p1 in poi_logtransmat.index:
        poi_logtransmat.loc[p1, p1] = 0
        rowsum = poi_logtransmat.loc[p1].sum()
        assert(rowsum > 0)
        logrowsum = np.log10(rowsum)
        for p2 in poi_logtransmat.columns:
            if p1 == p2:
                poi_logtransmat.loc[p1, p2] = -np.inf  # deal with log(0) explicitly
            else:
                poi_logtransmat.loc[p1, p2] = np.log10(poi_logtransmat.loc[p1, p2]) - logrowsum
    return poi_logtransmat

<a id='sec3.8'></a>

## 3.8 Recommendation based on POI Transition Matrix

Use dynamic programming to find a possibly non-simple path, i.e., walk.

In [ ]:
def find_path(V, E, ps, pe, L, withNodeWeight=False, alpha=0.5):
    assert(isinstance(V, pd.DataFrame))
    assert(isinstance(E, pd.DataFrame))
    assert(ps in V.index)
    assert(pe in V.index)
    # with sub-tours in trajectory, this is not the case any more, but it is nonsense to recommend such trajectories
    assert(2 < L <= V.index.shape[0])  
    if withNodeWeight == True:
        assert(0 < alpha < 1)
    beta = 1 - alpha
    
    A = pd.DataFrame(data=np.zeros((L-1, V.shape[0]), dtype=np.float), columns=V.index, index=np.arange(2, L+1))
    B = pd.DataFrame(data=np.zeros((L-1, V.shape[0]), dtype=np.int),   columns=V.index, index=np.arange(2, L+1))
    A += np.inf
    for v in V.index:
        if v != ps:
            if withNodeWeight == True:
                A.loc[2, v] = alpha * (V.loc[ps, 'weight'] + V.loc[v, 'weight']) + beta * E.loc[ps, v]  # ps--v
            else:
                A.loc[2, v] = E.loc[ps, v]  # ps--v
            B.loc[2, v] = ps
    
    for l in range(3, L+1):
        for v in V.index:
            if withNodeWeight == True: # ps-~-v1---v
                values = [A.loc[l-1, v1] + alpha * V.loc[v, 'weight'] + beta * E.loc[v1, v] for v1 in V.index]
            else:
                values = [A.loc[l-1, v1] + E.loc[v1, v] for v1 in V.index]  # ps-~-v1---v   
            
            minix = np.argmin(values)
            A.loc[l, v] = values[minix]
            B.loc[l, v] = V.index[minix]
            
    path = [pe]
    v = path[-1]
    l = L
    #while v != ps:  #incorrect if 'ps' happens to appear in the middle of a path
    while l >= 2:
        path.append(B.loc[l, v])
        v = path[-1]
        l -= 1
    path.reverse()
    return path

Use integer linear programming (ILP) to find a simple path.

In [ ]:
def find_path_ILP(V, E, ps, pe, L, withNodeWeight=False, alpha=0.5):
    assert(isinstance(V, pd.DataFrame))
    assert(isinstance(E, pd.DataFrame))
    assert(ps in V.index)
    assert(pe in V.index)
    assert(2 < L <= V.index.shape[0])
    if withNodeWeight == True:
        assert(0 < alpha < 1)
    beta = 1 - alpha
    
    p0 = str(ps); pN = str(pe); N = V.index.shape[0]
    
    # deal with np.inf which will cause ILP solver failure
    Edges = E.copy()
    INF = 1e6
    for p in Edges.index:
        Edges.loc[p, p] = INF 
    maxL = np.max(Edges.values.flatten())
    if maxL > INF:
        for p in Edges.index:
            Edges.loc[p, p] = maxL    
    
    # REF: pythonhosted.org/PuLP/index.html
    pois = [str(p) for p in V.index] # create a string list for each POI
    pb = pulp.LpProblem('MostLikelyTraj', pulp.LpMinimize) # create problem
    # visit_i_j = 1 means POI i and j are visited in sequence
    visit_vars = pulp.LpVariable.dicts('visit', (pois, pois), 0, 1, pulp.LpInteger) 
    # a dictionary contains all dummy variables
    dummy_vars = pulp.LpVariable.dicts('u', [x for x in pois if x != p0], 2, N, pulp.LpInteger)
    
    # add objective
    objlist = []
    if withNodeWeight == True:
        objlist.append(alpha * V.loc[int(p0), 'weight'])
    for pi in [x for x in pois if x != pN]:     # from
        for pj in [y for y in pois if y != p0]: # to
            if withNodeWeight == True:
                objlist.append(visit_vars[pi][pj] * (alpha*V.loc[int(pj), 'weight'] + beta*Edges.loc[int(pi), int(pj)]))
            else:
                objlist.append(visit_vars[pi][pj] * Edges.loc[int(pi), int(pj)])
    pb += pulp.lpSum(objlist), 'Objective'
    
    # add constraints, each constraint should be in ONE line
    pb += pulp.lpSum([visit_vars[p0][pj] for pj in pois if pj != p0]) == 1, 'StartAt_p0'
    pb += pulp.lpSum([visit_vars[pi][pN] for pi in pois if pi != pN]) == 1, 'EndAt_pN'
    if p0 != pN:
        pb += pulp.lpSum([visit_vars[pi][p0] for pi in pois]) == 0, 'NoIncoming_p0'
        pb += pulp.lpSum([visit_vars[pN][pj] for pj in pois]) == 0, 'NoOutgoing_pN'
    pb += pulp.lpSum([visit_vars[pi][pj] for pi in pois if pi != pN for pj in pois if pj != p0]) == L-1, 'Length'
    for pk in [x for x in pois if x not in {p0, pN}]:
        pb += pulp.lpSum([visit_vars[pi][pk] for pi in pois if pi != pN]) == \
              pulp.lpSum([visit_vars[pk][pj] for pj in pois if pj != p0]), 'ConnectedAt_' + pk
        pb += pulp.lpSum([visit_vars[pi][pk] for pi in pois if pi != pN]) <= 1, 'Enter_' + pk + '_AtMostOnce'
        pb += pulp.lpSum([visit_vars[pk][pj] for pj in pois if pj != p0]) <= 1, 'Leave_' + pk + '_AtMostOnce'
    for pi in [x for x in pois if x != p0]:
        for pj in [y for y in pois if y != p0]:
            pb += dummy_vars[pi] - dummy_vars[pj] + 1 <= (N - 1) * (1 - visit_vars[pi][pj]), \
                    'SubTourElimination_' + pi + '_' + pj
    #pb.writeLP("traj_tmp.lp")
    # solve problem
    pb.solve(pulp.PULP_CBC_CMD(options=['-threads', '6', '-strategy', '1', '-maxIt', '2000000'])) # CBC
    #gurobi_options = [('TimeLimit', '1800'), ('Threads', '10'), ('NodefileStart', '0.9'), ('Cuts', '2')]
    #pb.solve(pulp.GUROBI_CMD(options=gurobi_options)) # GUROBI
    visit_mat = pd.DataFrame(data=np.zeros((len(pois), len(pois)), dtype=np.float), index=pois, columns=pois)
    for pi in pois:
        for pj in pois: visit_mat.loc[pi, pj] = visit_vars[pi][pj].varValue

    # build the recommended trajectory
    recseq = [p0]
    while True:
        pi = recseq[-1]
        pj = visit_mat.loc[pi].idxmax()
        assert(round(visit_mat.loc[pi, pj]) == 1)
        recseq.append(pj); 
        #print(recseq); sys.stdout.flush()
        if pj == pN: return [int(x) for x in recseq]

In [ ]:
if run_tran == True:
    recdict_tran = dict()
    cnt = 0
    for i in range(len(trajid_set_all)):
        tid = trajid_set_all[i]
        t = extract_traj(tid, traj_all)
        
        # trajectory too short
        if len(t) < 3: continue
            
        # TSP, generally an ILP very hard for solvers (GUROBI/CBC)
        if t[0] == t[-1]: continue

        if uspecific == True:
            user = traj_user.loc[tid, 'userID']
            trajid_set_user = traj_user[traj_user['userID'] == user].index.tolist()
            trajid_set_other = traj_user[traj_user['userID'] != user].index.tolist()
            assert(tid in trajid_set_user)
            trajid_set_add = [x for x in trajid_set_user if x != tid]
            trajid_list_train = trajid_set_other.copy()
            assert(KX > 0)
            for k in range(KX):
                trajid_list_train = trajid_list_train + trajid_set_add
        else:
            trajid_list_train = trajid_set_all[:i] + trajid_set_all[i+1:]

        poi_info = calc_poi_info(trajid_list_train, traj_all, poi_all)
        
        # trajectory too long
        if len(t) > poi_info.shape[0]: continue
        
        print(t, '#%d ->' % cnt); cnt += 1; sys.stdout.flush()
        
        poi_logtransmat = gen_poi_logtransmat(trajid_list_train, traj_all, poi_info)
        nodes = poi_info.copy()
        edges = poi_logtransmat.copy()
        edges = -1 * edges  # edge weight is negative log of transition probability

        # DP
        rec_dp = find_path(nodes, edges, t[0], t[-1], len(t))

        # ILP
        rec_ilp = find_path_ILP(nodes, edges, t[0], t[-1], len(t))

        recdict_tran[tid] = {'REAL':t, 'REC_DP':rec_dp, 'REC_ILP':rec_ilp}
        print(' '*10, rec_dp)
        print(' '*10, rec_ilp); sys.stdout.flush()

In [ ]:
if run_tran == True:
    pickle.dump(recdict_tran, open(frecdict_tran, 'bw'))

In [ ]:
if run_tran == True:
    F11_tran = []
    F12_tran = []
    for tid in sorted(recdict_tran.keys()):
        F11_tran.append(calc_F1(recdict_tran[tid]['REAL'], recdict_tran[tid]['REC_DP']))
        F12_tran.append(calc_F1(recdict_tran[tid]['REAL'], recdict_tran[tid]['REC_ILP']))
    print('DP : %.3f, %.3f' % (np.mean(F11_tran), np.std(F11_tran)))
    print('ILP: %.3f, %.3f' % (np.mean(F12_tran), np.std(F12_tran)))

In [ ]:
# Glas
# user agnostic
#DP : 0.708, 0.184
#ILP: 0.705, 0.168
#
# user specific
#DP : 0.695, 0.182
#ILP: 0.691, 0.177

# Osak
# user agnostic
#DP : 0.707, 0.229
#ILP: 0.670, 0.209
#
# user specific
#DP : 0.706, 0.218
#ILP: 0.656, 0.198

<a id='sec4'></a>

# 4. Combine POI Ranking with Factorised Markov Chain

NOTE that:
- Ranking is over POIs (in POI space)
- Transition probabilities of Markov Chain of POI features (in POI feature space)

Two approaches to combine ranking with MC:
- recommend trajectory in POI space: 
  - transform transition matrix of feature vector into transition matrix of POIs,
  - recommend a sequence of POIs.
- recommend trajectory in POI feature space: 
  - transform ranking probability of POIs into ranking probability of POI features,
  - recommend a sequence of features, 
  - transform the squence of features into a sequence of POIs.

Here we implement the first approach, i.e., recommend trajectory in POI space.

Two methods for recommendation:
- find a simple path with maximum objective in a digraph. (by negating log probabilities, it can be reduced to a finding shortest path will $k$-edges, which is probably NP-hard as it is the same as TSP when $L=|P|$, dynamic programming can be used for non-simple path.)
- make greedy choice (of POI or POI feature vector) step-by-step

Here we use dynamic programming to find a possibly non-simple path and use ILP to find a simple path.

In [ ]:
if run_comb == True:
    recdict_comb = dict()
    cnt = 0
    for i in range(len(trajid_set_all)):
        tid = trajid_set_all[i]
        t = extract_traj(tid, traj_all)
        
        # trajectory too short
        if len(t) < 3: continue
        
        # TSP, generally an ILP very hard for solvers (GUROBI/CBC)
        if t[0] == t[-1]: continue

        if uspecific == True:
            user = traj_user.loc[tid, 'userID']
            trajid_set_user = traj_user[traj_user['userID'] == user].index.tolist()
            trajid_set_other = traj_user[traj_user['userID'] != user].index.tolist()
            assert(tid in trajid_set_user)
            trajid_set_add = [x for x in trajid_set_user if x != tid]
            trajid_list_train = trajid_set_other.copy()
            assert(KX > 0)
            for k in range(KX):
                trajid_list_train = trajid_list_train + trajid_set_add
        else:
            trajid_list_train = trajid_set_all[:i] + trajid_set_all[i+1:]

        poi_info = calc_poi_info(trajid_list_train, traj_all, poi_all)
        
        # trajectory too long
        if len(t) > poi_info.shape[0]: continue
        
        print(t, '#%d ->' % cnt); cnt += 1; sys.stdout.flush()
        
        train_df = gen_train_df(trajid_list_train, df_columns, traj_all, poi_info, query_id_dict)
        ranksvm = RankSVM(ranksvm_dir, useLinear=True)
        ranksvm.train(train_df)
        test_df = gen_test_df(t[0], t[-1], len(t), df_columns, poi_info, query_id_dict)
        rank_df = ranksvm.predict(test_df)
        rank_df.set_index('poiID', inplace=True)

        nodes = rank_df.copy()
        nodes['weight'] = -np.log(nodes['probability'])  # node weight is negative log of ranking probability
        nodes.drop('probability', axis=1, inplace=True)
        edges = poi_logtransmat.copy()
        edges = -1 * edges  # edge weight is negative log of transition probability

        # DP
        rec_dp = find_path(nodes, edges, t[0], t[-1], len(t), withNodeWeight=True, alpha=ALPHA)

        # ILP
        rec_ilp = find_path_ILP(nodes, edges, t[0], t[-1], len(t), withNodeWeight=True, alpha=ALPHA)

        recdict_comb[tid] = {'REAL':t, 'REC_DP':rec_dp, 'REC_ILP':rec_ilp} # save recommended trajectories
        print(' '*10, rec_dp)
        print(' '*10, rec_ilp); sys.stdout.flush()   

In [ ]:
if run_comb == True:
    pickle.dump(recdict_comb, open(frecdict_comb, 'bw'))

In [ ]:
if run_comb == True:
    F11_comb = []
    F12_comb = []
    for tid in sorted(recdict_comb.keys()):
        F11_comb.append(calc_F1(recdict_comb[tid]['REAL'], recdict_comb[tid]['REC_DP']))
        F12_comb.append(calc_F1(recdict_comb[tid]['REAL'], recdict_comb[tid]['REC_ILP']))
    print('DP : %.3f, %.3f' % (np.mean(F11_comb), np.std(F11_comb)))
    print('ILP: %.3f, %.3f' % (np.mean(F12_comb), np.std(F12_comb)))

In [ ]:
# Glas
# user agnostic
#DP : 0.745, 0.180
#ILP: 0.743, 0.179
#
# user specific
#DP : 0.714, 0.183
#ILP: 0.724, 0.167

# Osak
# user agnostic
#DP : 0.709, 0.202
#ILP: 0.660, 0.206
#
# user specific
#DP : 0.658, 0.192
#ILP: 0.635, 0.190

<a id='sec5'></a>

# 5. Comparison

Random guessing.

In [ ]:
from scipy.misc import comb
from math import factorial
def rand_guess(npoi, length):
    assert(length <= npoi)
    if length == npoi: return 1
    N = npoi - 2
    m = length - 2 # number of correct POIs
    k = m
    expected_F1 = 0
    while k >= 0:
        F1 = (k + 2) / length
        prob = comb(m, k) * comb(N-m, m-k) / comb(N, m)
        expected_F1 += prob * F1
        k -= 1
    return expected_F1

Sanity check.

In [ ]:
rand_guess(20, 5)